In [1]:
import pandas as pd
import numpy as np
from scipy import sparse

from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, StandardScaler, LabelEncoder
#from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin


from xgboost import XGBClassifier
import xgboost as xgb

from datetime import datetime
from datetime import date

import re
import pickle

In [2]:
from sklearn.neighbors import KNeighborsClassifier

In [3]:
from sklearn.metrics import accuracy_score, jaccard_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix

In [4]:
import nltk

In [5]:
from nltk.corpus import stopwords                   #Stopwords corpus
from nltk.tokenize import word_tokenize

In [6]:
# import textblob
# from textblob import TextBlob

In [7]:
from string import punctuation

In [8]:
import requests,json

import csv
from datetime import datetime

#### read train and test data

In [9]:
df_train = pd.read_csv('C:/Projects/FCSD/FNOP/old/Traindata_pilot_sample_v0.csv', engine='python')

In [10]:
df_test = pd.read_csv('C:/Projects/FCSD/FNOP/Data EDA/Testdata_pilot_sample_v0_top40parts.csv', engine='python')

### Use universal sentence encoder to create doc embedding

In [11]:
lookup_table = 'Misspellings_Correction_Synonyms_Lookup_Table_Master.xlsx'

In [12]:
syn_df=pd.read_excel(lookup_table)
syn_df = syn_df[syn_df['Review'] != "D"]
syn_dict=dict(zip(syn_df.misspelled, syn_df.corrected))

In [13]:
def TextPreprocess(text):
        # Avoid mixed data type errors and apply str on all rows
        # convert to lower case
        text = str(text).lower()

        #Remove HTML Tags
        text = re.sub('<[^>]*>', '', text)
        r1 = re.compile('<.*?>')
        text = re.sub(r1, ' ', text)

#         # Remove any token with numbers and/or special characters
#         r2 = re.compile(r'^[0-9\W\_]+$')
#         text = re.sub(r2, ' ', text)
        
        # move parenthesis content to the end of paragraph
        paren_content_extract = re.compile(r'\(([^()]+)\)')
        p_cont = paren_content_extract.findall(text)
        text = re.sub(r'\([^()]*\)', ' ', text)
        text = text + '. '.join(p_cont) + '.'
        
        # hard code to convert abbreviations with \'s
        text = re.sub(r"what's", "what is ", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", "have not ", text)
        text = re.sub(r"can't", "can not ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        text = re.sub(r"\'scuse", " excuse ", text)
        
        # if a token starts with numerical or # sign, remove it
        text = re.sub('\b[\d#]+\S*\b', ' ', text)
        # if a token ends with numberical, remove it
        text = re.sub(r'\b\S+\d+\b', ' ', text)

#         #Removing Punctuations
#         text = re.sub(r'[?|!|\'|"|#]',r'',text)
#         text = re.sub(r'[.|,|)|(|\|/]',r' ',text)     

        #Removing Punctuations
        text = re.sub(r'[\'|"|#]',r'',text)
        text = re.sub(r'[)|(]',r' ',text) 
        
        # whitespace characters are replaced into space, multiple dots replaced into single dot
        text = re.sub('\s+', ' ', text)
        text = re.sub(r'\.+', '.', text)

        # Tokenize all text data using nltk package
        text = word_tokenize(text)

#         # Remove stop words
#         stop = stopwords.words('english')
#         stop1 = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z', \
#                  'c/s','c/states','c/stated','cu', 'cs','c-s', 'cus','tomer', 'stomer', 'tate', 'cust',  'cust states', \
#                  'advise','nana','NaN','customer','states','rr','nan','wont','ck','check','pls','please', 'client','owner']

#         # Combine default nltk stop list and custom stop list to generate a deduplicated list
#         english_stops = list(set(stop+stop1))

#         text = [item for item in text if item not in english_stops]

#         # Remove special characters occuring as prefixes
#         text = [re.sub(r'^[\W\_]+','',str(i)) for i in text]
#         # Remove special characters occuring as suffixes
#         text = [re.sub(r'[\W\_]+$','',str(i)) for i in text]

#         # Replace special characters with spaces when occuring within the token 
#         #(i.e. not at the beginning or the end) only for tokens greater than 5 characters in length
#         text =[re.sub(r'[\W\_]+', ' ', str(i)) if len(str(i)) > 5 else i for i in text]

#         # Replace special characters with null when occuring within the token 
#         #(i.e. not at the beginning or the end) only for tokens less than 6 characters in length
#         text = [re.sub(r'[\W\_]+', '', str(i)) if len(str(i)) < 6 else i for i in text]

#         # Remove stop words after data cleansing steps to remove any residual stop words
#         text = [item for item in text if item not in english_stops]

#         # remove any numbers after data cleansing steps to remove any residual numeric values
#         r3 = re.compile(r'^[0-9\W\_]+$')
#         text = [i for i in text if not r3.match(i)]

        #Perform synonym lookup to find matching correctly spelled tokens for misspelled tokens
        #spell=SpellChecker() 
        text = [syn_dict.get(str(i), str(i)) for i in text]

        # Convert the list to string before passing to lemmatization module
        text = ' '.join(text)

#         # Perform Lemmatization on the cleaned up data before tfidf vectorization
#         text = lemmatize_with_postag(text)
        text = text.replace(' .', '.').replace(' ?', '?').replace(' !', '!').replace(' ,', ',').replace(' /', '/')
        return(text)

In [14]:
text_df = df_train[['cust_txt']]

In [15]:
text_df['process_txt']=text_df['cust_txt'].apply(lambda x: TextPreprocess(x))

C:\Users\jluo51\AppData\Local\Continuum\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
### convert null values of customer_txt into ' '
text_df.loc[text_df['process_txt'].isnull()==True,'process_txt'] = ' '

In [19]:
import tensorflow as tf
import tensorflow_hub as hub

In [20]:
### load the model
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
use_model = hub.KerasLayer(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [21]:
sentence_embeddings_X = use_model(text_df['process_txt']).numpy()

In [24]:
column_names = ['col'+str(i) for i in range(512)]

In [25]:
cust_text_embedding = pd.DataFrame(sentence_embeddings_X, columns = column_names)

#### create label encoded y variables

In [27]:
le = LabelEncoder()
le.fit(df_train['target'])
label_encoded_y = le.transform(df_train['target'])

### train with MLP model

In [28]:
from sklearn.neural_network import MLPClassifier

In [55]:
clf = MLPClassifier(solver='adam', activation='relu', learning_rate_init = 0.001, max_iter=100, batch_size=256, tol=1e-4, alpha=0.0001, hidden_layer_sizes=(10,10,10,10),random_state=1, verbose=True)

In [56]:
clf.fit(cust_text_embedding, label_encoded_y)

Iteration 1, loss = 2.48368215
Iteration 2, loss = 1.78624316
Iteration 3, loss = 1.67161541
Iteration 4, loss = 1.58639061
Iteration 5, loss = 1.53609035
Iteration 6, loss = 1.50417769
Iteration 7, loss = 1.48354713
Iteration 8, loss = 1.46828227
Iteration 9, loss = 1.45623311
Iteration 10, loss = 1.44456639
Iteration 11, loss = 1.43266451
Iteration 12, loss = 1.41774036
Iteration 13, loss = 1.39962337
Iteration 14, loss = 1.38235274
Iteration 15, loss = 1.36847494
Iteration 16, loss = 1.35669870
Iteration 17, loss = 1.34838280
Iteration 18, loss = 1.34107691
Iteration 19, loss = 1.33546870
Iteration 20, loss = 1.33031393
Iteration 21, loss = 1.32564447
Iteration 22, loss = 1.32189176
Iteration 23, loss = 1.31838378
Iteration 24, loss = 1.31502196
Iteration 25, loss = 1.31251814
Iteration 26, loss = 1.30982154
Iteration 27, loss = 1.30751178
Iteration 28, loss = 1.30544880
Iteration 29, loss = 1.30339147
Iteration 30, loss = 1.30167790
Iteration 31, loss = 1.29968272
Iteration 32, los

C:\Users\jluo51\AppData\Local\Continuum\Anaconda3\envs\tf\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10, 10, 10, 10), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=100,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

#### create X_test and y_test

In [31]:
text_df_test = df_test[['cust_txt']]

In [32]:
text_df_test['process_txt']=text_df_test['cust_txt'].apply(lambda x: TextPreprocess(x))

C:\Users\jluo51\AppData\Local\Continuum\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
### convert null values of customer_txt into ' '
text_df_test.loc[text_df_test['process_txt'].isnull()==True,'process_txt'] = ' '

In [37]:
sentence_embeddings_X_test = use_model(text_df_test['process_txt']).numpy()

In [38]:
cust_text_embedding_test = pd.DataFrame(sentence_embeddings_X_test, columns = column_names)

#### create label encoded y variables for the test set

In [40]:
label_encoded_y_test = le.transform(df_test['target'])

#### score the test set with the trained knn model

In [57]:
y_pred = clf.predict(cust_text_embedding_test)

In [58]:
accuracy_score(label_encoded_y_test, y_pred)

0.6796643574522323